In [14]:
import geopandas as gpd
import folium
import json

# Ler o arquivo KML
meu_mapa = gpd.read_file('./meu_mapa.kml', driver='KML')

# Salvar como GeoJSON
meu_mapa.to_file('meu_mapa.geojson', driver='GeoJSON')

# Calcular a caixa delimitadora
bounds = meu_mapa.total_bounds  # Retorna [minx, miny, maxx, maxy]

# Criar o mapa com nome específico para referência no JavaScript
fig = folium.Map(location=[(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2], zoom_start=10, width='100%', height='100%', control_scale=True)

# Carregar o GeoJSON
geo_json_mapa = json.load(open('meu_mapa.geojson'))

# Adicionar o Choropleth
folium.Choropleth(
    geo_data=geo_json_mapa,
    fill_color="steelblue",
    fill_opacity=0.4,
    line_color="black",
    line_opacity=0.9
).add_to(fig)

# Ajustar o zoom para encaixar a área delimitada
fig.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Adicionar um marcador para cliques do usuário
folium.LatLngPopup().add_to(fig)

# Adicionar JavaScript para armazenar e desenhar os 4 pontos
js_code = """
    var points = [];
    function onMapClick(e) {
        points.push([e.latlng.lat, e.latlng.lng]);
        L.marker([e.latlng.lat, e.latlng.lng]).addTo(map);
        
        if (points.length === 4) {
            var polygon = L.polygon(points, {color: 'blue'}).addTo(map);
            map.fitBounds(polygon.getBounds());
            points = []; // Limpa os pontos para reiniciar
        }
    }

    // Vincular evento ao mapa
    var mapInstance = document.querySelector('div#map');  // Nome correto para o mapa Folium
    map.on('click', onMapClick);
"""

# Injetar o JavaScript no HTML gerado
fig.get_root().html.add_child(folium.Element(f"<script>{js_code}</script>"))

# Salvar o mapa interativo em HTML
fig.save('mapa_interativo_com_poligono.html')
